In [3]:

API_KEY = "CG-Hv2ecTjD5Pz6P1XURRje2kvA"

In [5]:
import os
import requests
import pandas as pd
from datetime import datetime, timezone

# =========================
# CONFIGURACIÓN
# =========================
#API_KEY = os.getenv("COINGECKO_API_KEY") dejalo comentado por que esta declarado arriba
BASE_URL = "https://pro-api.coingecko.com/api/v3"
HEADERS = {"accept": "application/json", "x-cg-pro-api-key": API_KEY}

START_DATE = pd.Timestamp("2018-01-01")  # ahora como Timestamp

OUT_BTC_DOM   = "btc_dominance.csv"
OUT_TOTAL_MC  = "total_mcap.csv"
OUT_STABLECAP = "stablecoin_supply.csv"
OUT_FNG       = "fear_greed.csv"

# =========================
# FUNCIONES AUXILIARES
# =========================
def save_series(series: pd.Series, path: str):
    series.index = pd.to_datetime(series.index)  # aseguramos tipo datetime
    df = series.reset_index()
    df.columns = ["Date", "value"]
    df = df[df["Date"] >= START_DATE]  # filtramos correctamente
    df.to_csv(path, index=False)
    print(f"✅ {path} ({len(df)} filas)  {df['Date'].min()} → {df['Date'].max()}")

def get_json(endpoint, params=None):
    url = f"{BASE_URL}{endpoint}"
    r = requests.get(url, headers=HEADERS, params=params, timeout=60)
    r.raise_for_status()
    return r.json()

def pairs_to_series(pairs):
    return pd.Series({
        datetime.fromtimestamp(ts/1000, tz=timezone.utc).date(): val
        for ts, val in pairs
    }).sort_index()

# =========================
# 1) TOTAL MARKET CAP
# =========================
j = get_json("/global/market_cap_chart", params={"vs_currency": "usd", "days": "max"})
pairs = j.get("market_cap_chart", {}).get("market_cap") or j.get("market_cap")
total_cap = pairs_to_series(pairs).asfreq("D").ffill()
save_series(total_cap, OUT_TOTAL_MC)

# =========================
# 2) BTC DOMINANCE
# =========================
btc_data = get_json("/coins/bitcoin/market_chart", params={"vs_currency": "usd", "days": "max"})
btc_cap = pairs_to_series(btc_data["market_caps"]).asfreq("D").ffill()
btc_dom = (btc_cap / total_cap) * 100
save_series(btc_dom, OUT_BTC_DOM)

# =========================
# 3) STABLECOIN SUPPLY (USDT)
# =========================
usdt_data = get_json("/coins/tether/market_chart", params={"vs_currency": "usd", "days": "max"})
usdt_cap = pairs_to_series(usdt_data["market_caps"]).asfreq("D").ffill()
save_series(usdt_cap, OUT_STABLECAP)

# =========================
# 4) FEAR & GREED (Alternative.me)
# =========================
fng_url = "https://api.alternative.me/fng/?limit=0&format=json"
r = requests.get(fng_url, timeout=60)
r.raise_for_status()
j = r.json()
df_fng = pd.DataFrame(j["data"])
df_fng["Date"] = pd.to_datetime(df_fng["timestamp"], unit="s", utc=True).dt.tz_convert(None)
fear_greed = df_fng.groupby("Date")["value"].last().astype(float).sort_index()
save_series(fear_greed, OUT_FNG)


✅ total_mcap.csv (2784 filas)  2018-01-01 00:00:00 → 2025-08-15 00:00:00
✅ btc_dominance.csv (2784 filas)  2018-01-01 00:00:00 → 2025-08-15 00:00:00
✅ stablecoin_supply.csv (2784 filas)  2018-01-01 00:00:00 → 2025-08-15 00:00:00
✅ fear_greed.csv (2749 filas)  2018-02-01 00:00:00 → 2025-08-15 00:00:00


/tmp/ipykernel_11603/135660624.py:74: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_fng["Date"] = pd.to_datetime(df_fng["timestamp"], unit="s", utc=True).dt.tz_convert(None)
